### Instalar as Dependências


In [1]:
pip install folium numpy tqdm

Note: you may need to restart the kernel to use updated packages.


### Importar Bibliotecas Necessárias


In [2]:
import folium
import random
import numpy as np
from tqdm import tqdm
import time


### Gerar Dados Aleatórios

In [3]:
# Coordenadas iniciais de BH
map_center = [-19.9208, -43.9378]

# Função para gerar coordenadas aleatórias espaçadas em torno de um ponto central
def generate_random_location(center, spread=0.05):
    return [center[0] + random.uniform(-spread, spread), center[1] + random.uniform(-spread, spread)]

# Gerar coordenadas aleatórias espaçadas para alunos
num_students = 10
students = [{'name': f'Aluno {i+1}', 'location': generate_random_location(map_center, spread=0.03)} for i in range(num_students)]

# Gerar coordenadas aleatórias para a escola
school = {'name': 'Escola', 'location': generate_random_location(map_center, spread=0.03)}

# Gerar coordenadas aleatórias para a garagem
garage = {'name': 'Garagem', 'location': generate_random_location(map_center, spread=0.03)}

### Definir Funções para a Computação Evolucionária

#### Função de Cálculo de Distância

In [4]:
# Função simples, já que não trabalharemos com rotas nas ruas em si, e sim com uma aproximação pelo deslocamento
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

#### Função de fitness

In [5]:
def fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += calculate_distance(route[i], route[i + 1])
    return total_distance

#### Função de criação de rotas

In [6]:
def create_route(garage, students, school):
    student_locations = [student['location'] for student in students]
    random.shuffle(student_locations)
    stops = [garage['location']] + student_locations + [school['location']]
    return stops

#### Função de mutação

In [7]:
def mutate(route):
    student_positions = route[1:-1]  # Alunos estão entre a garagem e a escola
    idx1, idx2 = random.sample(range(len(student_positions)), 2)
    student_positions[idx1], student_positions[idx2] = student_positions[idx2], student_positions[idx1]
    return [route[0]] + student_positions + [route[-1]]

#### Algoritmo Genético

In [12]:
def genetic_algorithm(garage, students, school, population_size=10000, generations=10000, mutation_rate=0.1):
    population = [create_route(garage, students, school) for _ in range(population_size)]
    best_route = min(population, key=fitness)
    
    for generation in tqdm(range(generations), desc="Calculando rotas"):
        new_population = []
        for route in population:
            if random.random() < mutation_rate:
                new_route = mutate(route[:])
                new_population.append(new_route)
            else:
                new_population.append(route)
        
        best_route_in_population = min(new_population, key=fitness)
        if fitness(best_route_in_population) < fitness(best_route):
            best_route = best_route_in_population
        
        population = new_population
    
    return best_route

### Visualização os Resultados

In [13]:
# Medir o tempo de execução
start_time = time.time()

# Gerar a melhor rota usando o algoritmo genético
best_route = genetic_algorithm(garage, students, school)

end_time = time.time()
execution_time = end_time - start_time
total_distance = fitness(best_route)

print(f"Tempo de execução: {execution_time:.2f} segundos")
print(f"Distância total da rota: {total_distance:.2f} unidades")

# Criar um novo mapa para adicionar as rotas
school_map_with_routes = folium.Map(location=map_center, zoom_start=12)

# Adicionar marcadores para alunos no novo mapa com a posição na rota
for i, stop in enumerate(best_route[1:-1], start=1):  # Exclui a garagem (primeira) e a escola (última)
    folium.Marker(location=stop, tooltip=f'Parada {i}', icon=folium.DivIcon(html=f'<div style="font-size: 12pt">{i}</div>')).add_to(school_map_with_routes)

for student in students:
    folium.Marker(location=student['location'], tooltip=student['name'], icon=folium.Icon(color='blue', icon='info-sign')).add_to(school_map_with_routes)


# Adicionar marcador para a escola no novo mapa
folium.Marker(location=school['location'], tooltip=school['name'], icon=folium.Icon(color='green', icon='school')).add_to(school_map_with_routes)

# Adicionar marcador para a garagem no novo mapa
folium.Marker(location=garage['location'], tooltip=garage['name'], icon=folium.Icon(color='red', icon='cog')).add_to(school_map_with_routes)

# Adicionar linha representando a rota no mapa
folium.PolyLine(best_route, color='red', weight=2.5, opacity=1).add_to(school_map_with_routes)

# Adicionar grid ao mapa usando JavaScript
grid_js = '''
<script>
L.GridLayer.DebugCoords = L.GridLayer.extend({
    createTile: function (coords) {
        var tile = document.createElement('div');
        tile.innerHTML = [coords.x, coords.y, coords.z].join(', ');
        tile.style.outline = '1px solid red';
        return tile;
    }
});

L.gridLayer.debugCoords = function(opts) {
    return new L.GridLayer.DebugCoords(opts);
}

L.gridLayer.debugCoords().addTo(map);
</script>
'''

school_map_with_routes.get_root().html.add_child(folium.Element(grid_js))

# Salvar o mapa com rotas em um arquivo HTML
school_map_with_routes.save('school_routes_map.html')

# Exibir o mapa com rotas no Jupyter Notebook (se estiver usando Jupyter)
school_map_with_routes

Calculando rotas: 100%|█| 10000/10000 [

Tempo de execução: 602.58 segundos
Distância total da rota: 0.18 unidades
